This notebook was created by [Philip Loche](https://github.com/PicoCentauri)

In [1]:
import os
from os.path import basename, splitext
from typing import List, Union

import ase.io
import numpy as np
from ase.build import molecule
from ase.calculators.cp2k import CP2K
from numpy.testing import assert_allclose

In [3]:
frames = []
frames += ase.io.read("sel_frames_250_32.xyz",":")

# Create input

In [2]:
def write_reftraj(fname: str, frames: Union[ase.Atoms, List[ase.Atoms]]):
    if isinstance(frames, ase.Atoms):
        frames = [frames]

    out = ""
    for i, atoms in enumerate(frames):
        if (
            len(atoms) != len(frames[0])
            or atoms.get_chemical_formula() != frames[0].get_chemical_formula()
        ):
            raise ValueError(
                f"Atom symbols in frame {i},{atoms.get_chemical_formula()} are "
                f"different compared to inital frame "
                f"{frames[0].get_chemical_formula()}. "
                "CP2K does not support changing atom types within a reftraj run!"
            )
        n_atoms = len(atoms)
        out += f"{len(atoms):>8}\n i = {i+1:>8}, time = {0:>12.3f}\n"
        for atom in atoms:
            pos = atom.position
            out += f"{atom.symbol}{pos[0]:24.15f}{pos[1]:24.15f}{pos[2]:24.15f}\n"
    out += "\n"
    with open(fname, "w") as f:
        f.write(out)

In [11]:
def write_cellfile(fname: str, frames: Union[ase.Atoms, List[ase.Atoms]]):
    if isinstance(frames, ase.Atoms):
        frames = [frames]

    out = "#   Step   Time [fs]       Ax [Angstrom]       Ay [Angstrom]       Az [Angstrom]       Bx [Angstrom]       By [Angstrom]       Bz [Angstrom]       Cx [Angstrom]       Cy [Angstrom]       Cz [Angstrom]      Volume [Angstrom^3]\n"
    for i, atoms in enumerate(frames):
        out += f"{i+1:>8}{0:>12.3f}"
        out += "".join([f"{c:>20.10f}" for c in atoms.cell.flatten()])
        out += f"{atoms.cell.volume:>25.10f}"
        out += "\n"

    with open(fname, "w") as f:
        f.write(out)

In [4]:
def write_cp2k_in(fname: str, project: str, last_snapshot: int, cell: List[float]):
    with open("reftraj_template.cp2k", "r") as f:
        cp2k_in = f.read()

    cp2k_in = cp2k_in.replace("//PROJECT//", project)
    cp2k_in = cp2k_in.replace("//LAST_SNAPSHOT//", str(last_snapshot))
    cp2k_in = cp2k_in.replace("//CELL//", " ".join([f"{c:.6f}" for c in cell]))

    with open(fname, "w") as f:
        f.write(cp2k_in)

In [5]:
def mkdir_force(*args, **kwargs):
    try:
        os.mkdir(*args, **kwargs)
    except OSError as e:
        pass

In [6]:
project = "NACL_v2"
project_directory = "production_H2O"
reference_file = "reference_structures/sel_frames_256.xyz"

In [9]:
frames_full = frames #ase.io.read(reference_file, ":")

frames_dict = {}

for atoms in frames_full:
    chemical_formula = atoms.get_chemical_formula()
    try:
        frames_dict[chemical_formula]
    except KeyError:
        frames_dict[chemical_formula] = []

    frames_dict[chemical_formula].append(atoms)

In [12]:
mkdir_force(project_directory)

for stoichiometry, frames in frames_dict.items():
    current_directory = f"{project_directory}/{stoichiometry}"
    mkdir_force(current_directory)

    write_cp2k_in(
        f"{current_directory}/in.cp2k",
        project=project,
        last_snapshot=len(frames),
        cell=frames[0].cell.diagonal(),
    )

    ase.io.write(f"{current_directory}/init.xyz", frames[0])
    write_reftraj(f"{current_directory}/reftraj.xyz", frames)
    write_cellfile(f"{current_directory}/reftraj.cell", frames)

# Load results

In [1]:
cflength = 0.529177210903  # Bohr -> Å
cfenergy = 27.211386245988  # Hartree -> eV
cfforce = cfenergy / cflength  # Hartree/Bohr -> eV/Å

In [10]:
new_frames = []

for stoichiometry, frames in frames_dict.items():
    current_directory = f"{project_directory}/{stoichiometry}"

    frames_dft = ase.io.read(f"{current_directory}/{project}-pos-1.xyz", ":")
    forces_dft = ase.io.read(f"{current_directory}/{project}-frc-1.xyz", ":")
    cell_dft = np.loadtxt(f"{current_directory}/{project}-1.cell")[:, 2:-1]

    for i_atoms, atoms in enumerate(frames_dft):
        frames_ref = frames[i_atoms]

        # Check consistent positions
        if not np.allclose(atoms.positions, frames_ref.positions):
            raise ValueError(f"Positions in frame {i_atoms} are not the same.")

        # Check consistent cell
        if not np.allclose(frames_ref.cell.flatten(), cell_dft[i_atoms]):
            raise ValueError(f"Cell dimensions in frame {i_atoms} are not the same.")

        atoms.info["E"] *= cfenergy
        atoms.pbc = True
        atoms.cell = frames_ref.cell
        atoms.set_array("forces", cfforce * forces_dft[i_atoms].positions)

    new_frames += frames_dft

new_fname = f"{splitext(basename(reference_file))[0]}_dft.xyz"
ase.io.write(f"{project_directory}/{new_fname}", new_frames)